# **K Nearest Neighbors**

K-Nearest Neighbors (KNN) is a classification algorithm. The central idea is that data points with similar attributes tend to fall into similar categories. If you have a dataset of points where the class of each point is known, you can take a new point with an unknown class, find it’s nearest neighbors, and classify it.

## **First Principles**

### **Distance Formula**

#### **Distance Between Points - 2D**

We need to define what it means for two points to be close together or far apart. To do this, we’re going to use the Distance Formula.

In [1]:
star_wars = [125, 1977]
raiders = [115, 1981]
mean_girls = [97, 2004]


def distance(movie1, movie2):
    length_difference = (movie1[0] - movie2[0]) ** 2
    year_difference = (movie1[1] - movie2[1]) ** 2
    distance = (length_difference + year_difference) ** 0.5
    return distance


print(distance(star_wars, raiders))
print(distance(star_wars, mean_girls))

10.770329614269007
38.897300677553446


#### **Distance Between Points - nth Dimensional**

The generalized distance formula between points A and B is as follows:

$\sqrt{(A_1-B_1)^2+(A_2-B_2)^2+...+(A_n-B_n)^2}$

Here, $A_1-B_1$ is the difference between the first feature of each point. $A_n-B_n$ is the difference between the last feature of each point. Using this formula, we can find the K-Nearest Neighbors of a point in N-dimensional space

In [2]:
star_wars = [125, 1977, 11000000]
raiders = [115, 1981, 18000000]
mean_girls = [97, 2004, 17000000]


def distance(movie1, movie2):
    squared_difference = 0
    for i in range(len(movie1)):
        squared_difference += (movie1[i] - movie2[i]) ** 2
    final_distance = squared_difference**0.5
    return final_distance


print(distance(star_wars, raiders))
print(distance(star_wars, mean_girls))

7000000.000008286
6000000.000126083


### **Min-Max Normalisation**

The problem is that the distance formula treats all dimensions equally, regardless of their scale. The solution to this problem is to normalize the data so every value is between 0 and 1. In this lesson, we’re going to be using min-max normalization.

In [3]:
release_dates = [
    1897.0,
    1998.0,
    2000.0,
    1948.0,
    1962.0,
    1950.0,
    1975.0,
    1960.0,
    2017.0,
    1937.0,
    1968.0,
    1996.0,
    1944.0,
    1891.0,
    1995.0,
    1948.0,
    2011.0,
    1965.0,
    1891.0,
    1978.0,
]


def min_max_normalize(lst):
    minimum = min(lst)
    maximum = max(lst)
    normalized = []

    for value in lst:
        normalized_num = (value - minimum) / (maximum - minimum)
        normalized.append(normalized_num)

    return normalized


print(min_max_normalize(release_dates))

[0.047619047619047616, 0.8492063492063492, 0.8650793650793651, 0.4523809523809524, 0.5634920634920635, 0.46825396825396826, 0.6666666666666666, 0.5476190476190477, 1.0, 0.36507936507936506, 0.6111111111111112, 0.8333333333333334, 0.42063492063492064, 0.0, 0.8253968253968254, 0.4523809523809524, 0.9523809523809523, 0.5873015873015873, 0.0, 0.6904761904761905]


### **Classification Process**

Now that our data has been normalized and we know how to find the distance between two points, we can begin classifying unknown data!

To do this, we want to find the k nearest neighbors of the unclassified point. In a few exercises, we’ll learn how to properly choose k, but for now, let’s choose a number that seems somewhat reasonable. Let’s choose 5.

```python
def classify(unknown, dataset, k=5):
  distances = []
  #Looping through all points in the dataset
  for title in dataset:
    movie = dataset[title]
    distance_to_point = distance(movie, unknown)
    #Adding the distance and point associated with that distance
    distances.append([distance_to_point, title])
  distances.sort()
  #Taking only the k closest points
  neighbors = distances[0:k]
```

In order to find the 5 nearest neighbors, we need to compare this new unclassified movie to every other movie in the dataset. This means we’re going to be using the distance formula again and again. We ultimately want to end up with a sorted list of distances and the movies associated with those distances.

```python
[
  [0.083, 'Lady Vengeance'],
  [0.236, 'Steamboy'],
  ...
  ...
  [0.331, 'Godzilla 2000']
]
```

Our goal now is to count the number of good movies and bad movies in the list of neighbors. If more of the neighbors were good, then the algorithm will classify the unknown movie as good. Otherwise, it will classify it as bad.

In order to find the class of each of the labels, we’ll need to look at our movie_labels dataset. For example, `movie_labels['Akira']` would give us 1 because Akira is classified as a good movie.

```python
def classify(unknown, dataset, labels, k):
  distances = []
  #Looping through all points in the dataset
  for title in dataset:
    movie = dataset[title]
    distance_to_point = distance(movie, unknown)
    #Adding the distance and point associated with that distance
    distances.append([distance_to_point, title])
  distances.sort()
  #Taking only the k closest points
  neighbors = distances[0:k]
  num_good = 0
  num_bad = 0
  for neighbor in neighbors:
    title = neighbor[1]
    if labels[title] == 0:
      num_bad += 1
    elif labels[title] == 1:
      num_good += 1
  if num_good > num_bad:
    return 1
  else:
    return 0
```

### **Choosing K**

The validation accuracy changes as k changes. The first situation that will be useful to consider is when k is very small. Let’s say `k = 1`. We would expect the validation accuracy to be fairly low due to overfitting. Overfitting is a concept that will appear almost any time you are writing a machine learning algorithm. Overfitting occurs when you rely too heavily on your training data; you assume that data in the real world will always behave exactly like your training data. In the case of K-Nearest Neighbors, overfitting happens when you don’t consider enough neighbors. A single outlier could drastically determine the label of an unknown point. 

On the other hand, if k is very large, our classifier will suffer from underfitting. Underfitting occurs when your classifier doesn’t pay enough attention to the small quirks in the training set. Imagine you have 100 points in your training set and you set `k = 100`. Every single unknown point will be classified in the same exact way. The distances between the points don’t matter at all! This is an extreme example, however, it demonstrates how the classifier can lose understanding of the training data if k is too big.

## **Model Fitting**

However, rather than writing your own classifier every time, you can use Python’s `sklearn` library. 

### **sklearn.neighbors.KNeighborsClassifier**

```python
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=9)

# finds the coefficients and the intercept value
knn.fit(X_train, y_train)

y_predicted = knn.predict(X_test)
```

### **sklearn.neighbors.KNeighborsRegressor**

The K-Nearest Neighbors algorithm is a powerful supervised machine learning algorithm typically used for classification. However, it can also perform regression. We can compute a weighted average based on how close each neighbor is. Let’s say we’re trying to predict the rating of movie X and we’ve found its three nearest neighbors. Consider the following table:

|Movie|Rating|Distance to Movie X|
|-|-|-|
|A|5.0|3.2|
|B|6.8|11.5|
|C|9.0|1.1|

If we find the mean, the predicted rating for X would be 6.93. However, movie X is most similar to movie C, so movie C’s rating should be more important when computing the average. Using a weighted average, we can find movie X’s rating. The numerator is the sum of every rating divided by their respective distances. The denominator is the sum of one over every distance. Even though the ratings are the same as before, the weighted average has now gone up to 7.9.

The `KNeighborsRegressor` class is very similar to `KNeighborsClassifier`. We can also choose whether or not to use a weighted average using the parameter weights. If weights equals "uniform", all neighbors will be considered equally in the average. If weights equals "distance", then a weighted average is used.

```python
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor(n_neighbors=5, weights="distance")

knn_reg.fit(X_train, y_train)
```

## **Accuracy Curve**

```python
k_values = list(range(1, 100))
k_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    k_scores.append(knn.score(X_test, y_test))

sns.lineplot(x=k_values, y=k_scores)
```